# Subthreshold response to somatic current injection is close to linear 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell

np.random.seed(12345)

In [ ]:
def insert_current_stimuli(amp, cell, delay):

    stim_params = {'amp': amp,
                   'idx': 0,
                   'pptype': "IClamp",
                   'dur': 1e9,
                   'delay': delay}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

current_amps = np.linspace(-0.45, 0.45, 15)  # 118 221,

tstop = 50
dt = 2**-5
delay = 5
dVs = np.zeros(len(current_amps))
somavs = []
tvec = None

for i, amp in enumerate(current_amps):
    cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
    syn, cell = insert_current_stimuli(amp, cell, delay=delay)
    cell.simulate(rec_imem=False, rec_vmem=True)
    v = cell.vmem[0, :]
    t0_idx = np.argmin(np.abs(cell.tvec - delay)) - 1
    dV_max_idx = np.argmax(np.abs(v - v[t0_idx]))
    dVs[i] = v[dV_max_idx] - v[t0_idx]
    somavs.append(v.copy())
    tvec = cell.tvec.copy()
    del cell
    del syn


In [ ]:
plt.close("all")
fig = plt.figure(figsize=[6, 2])
fig.subplots_adjust(bottom=0.17, top=0.85, right=0.99,
                    left=0.1, wspace=0.3)


ax1 = fig.add_subplot(121, xlabel="time (ms)",
                      ylabel=r"somatic $V_{\rm m}$ (mV)")
ax2 = fig.add_subplot(122, xlabel=r"$I_{\rm inj}$ (nA)", ylabel=r"max $\Delta V_{\rm m}$ (mV)")
for i, amp in enumerate(current_amps):
    ax1.plot(tvec, somavs[i], 'k', lw=1)

ax2.plot(current_amps, dVs, 'k-o', ms=5, lw=1)
ax2.plot([current_amps[0], current_amps[-2]], [dVs[0], dVs[-2]], 'r--')

mark_subplots(fig.axes)
simplify_axes(fig.axes)
fig.savefig("fig_hay_subthresh_linearity.pdf")
